### Initial Setting

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=fc97bbe5681e2ba33534d49e368be7120d25036ce3db38a123c8b9c5e2c634db
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [5]:
!pip install umap-learn

     |████████████████████████████████| 80 kB 4.7 MB/s 
     |████████████████████████████████| 1.1 MB 24.6 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-py3-none-any.whl size=76566 sha256=f455fc843adffa39c59d5e0593f60c0b2b1022e9b55b61c2f9901ddf705a08a2
  Stored in directory: /root/.cache/pip/wheels/01/e7/bb/347dc0e510803d7116a13d592b10cc68262da56a8eec4dd72f
  Created wheel for pynndescent: filename=pynndescent-0.5.4-py3-none-any.whl size=52372 sha256=307ee54686a974804dd25fbcacb59d30e1cd20436967d59432ef5ca9d80c2cf6
  Stored in directory: /root/.cache/pip/wheels/d0/5b/62/3401692ddad12324249c774c4b15ccb046946021e2b581c043
Successfully built umap-learn pynndescent


In [107]:
from efficientnet_pytorch import EfficientNet
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import os
import glob
import logging
import torch
from torch import nn
import pickle
from tqdm import tqdm # progress bar
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import plot

In [88]:
# dataset path
data_path = "/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/"
# cluster_list.txt path
cluster_list_path = "/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_lee/cluster_list/cluster_list_test_10.txt"
# feature path
feat_path = "/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_lee/features/features_lee_test_10.npy"
# filename path
filename_path = "/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_lee/filenames/filenames_lee_test_10.npy"
# textfile path
textfile_path = "/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_lee/result.txt"

In [100]:
# make data_list : .jpg files
data_list = []
data_list = [files for files in glob.glob(data_path + '*/*.jpg')]
# make cluster_list : cluster label name files
cluster_list = open(cluster_list_path, 'r').read().split('\n')
# make option_list
option_list = {'feature':'features_lee_test_10'}

['/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_0.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_2.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_5.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_7.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_16.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_17.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_18.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_20.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Together/Dataset/Filtered_img/circle/circle_28.jpg', '/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_Togeth

### Save Images

In [ ]:
# save images
images = np.load("/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_lee/data/test/full_numpy_bitmap_mouth.npy")
os.chdir("/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_lee/data/test")
for i in range(10000):
  image = Image.fromarray(images[i].reshape(28, 28))
  image.save(str(i) + '.jpg')

### Rename Files

In [36]:
# rename files
def ChangeName(path, name):
    i = 0
    for filename in os.listdir(path):
        os.rename(path+filename, path+str(name)+'_'+str(i)+'.jpg')
        i += 1
ChangeName(data_path + "mouth/","mouth")

### Feature Extraction

In [55]:
# transform
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),]) # compose image (resize, totensor, normalize)
pooling = nn.AdaptiveAvgPool2d(1)

def ExtractFeature(image, model):
    img = tfms(Image.open(image).convert("RGB")).unsqueeze(0) # unsqueeze: vector to tensor
    features = model.extract_features(img)
    features = pooling(features)
    return features

In [ ]:
#efficientNet
#feature extraction
model = EfficientNet.from_pretrained('efficientnet-b0')
data = {}
for i, image in enumerate(tqdm(data_list)): # i for tuple data type
  feat = ExtractFeature(image, model)
  feat = feat.detach().numpy().reshape(-1) # detach: copy tensor without gradient
  data[image] = (feat)
np.save(feat_path, np.array(list(data.values()))) # get a list of the features
np.save(filename_path, np.array(list(data.keys()))) # get a list of the filenames

### Dimension Reduction

In [ ]:
feat = np.load(feat_path)

In [102]:
# PCA
from sklearn.decomposition import PCA
option_list['reduction'] = 'PCA'
option_list['PCAComponents'] = 2
pca = PCA(n_components=option_list['PCAComponents'], random_state=0)
pca.fit(feat)
x = pca.transform(feat)

In [111]:
# TSNE
from sklearn.manifold import TSNE
option_list['reduction'] = 'TSNE'
option_list['TSNEComponents'] = 3
tsne = TSNE(n_components=option_list['TSNEComponents'], init='pca', random_state=0)
x = tsne.fit_transform(feat)

In [ ]:
# UMAP
import umap
option_list['reduction'] = 'UMAP'
option_list['UMAPComponents'] = 2
umap = umap.UMAP(n_components=option_list['UMAPComponents'], random_state=0)
umap.fit(feat)
x = umap.transform(feat)

### Clustering

In [112]:
def Cluster(cmd, cluster_num, input):
  if cmd == 'kmeans':
    from sklearn.cluster import KMeans
    option_list['clustering'] = 'KMeans'
    kmeans = KMeans(n_clusters = cluster_num, n_jobs = -1, random_state = 0)
    kmeans.fit(input)
    return kmeans.labels_
  elif cmd == 'dbscan':
    from sklearn.cluster import DBSCAN
    option_list['clustering'] = 'DBSCAN'
    option_list['DBEps'] = 1.3
    option_list['DBMinSamples'] = 2
    dbscan = DBSCAN(eps=option_list['DBEps'], min_samples=option_list['DBMinSamples'])
    dbscan.fit(input)
    return dbscan.labels_
  elif cmd == 'birch':
    from sklearn.cluster import Birch
    option_list['clustering'] = 'BIRCH'
    brc = Birch(n_clusters=cluster_num)
    brc.fit(input)
    return brc.labels_

In [121]:
labels = Cluster('birch', len(cluster_list), x)

In [ ]:
os.chdir("/content/drive/MyDrive/Colab/Sketch_RNN/Sketch_RNN_lee")
fig = px.scatter(x, x=0, y=1, color = labels)
plot(fig)

### Testing

In [ ]:
filenames = np.load(filename_path)
groups = {}
# holds the cluster id and the images { id: [images] }
for f, cluster in zip(filenames, labels):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(f)
    else:
        groups[cluster].append(f)
print(groups)

# make cluster_dict for calculate acc
cluster_dict = {}
for cluster in groups:
  image_count = []
  image_name = []
  for image in groups[cluster]:
    image_name.append(image.split('/')[-2])
  for name in cluster_list:
    image_count.append(image_name.count(name))
  cluster_dict[cluster] = cluster_list[image_count.index(max(image_count))] # Select the most frequently classified label as the correct label
option_list['clusterDictSize'] = len(cluster_dict)
for i in range(len(cluster_dict) - 1):
  print(str(cluster_dict[i]) + ' : ' + str(len(groups[i])))

# get accuracy
ans = []
result = []    
for cluster in groups:
    for category in groups[cluster]:
        ans.append(cluster_dict[cluster])
        result.append(category.split('/')[-2])
acc = str(round(f1_score(myans, ans, average='micro') * 100, 2)) + '%'
option_list['accuracy'] = acc
print(acc)

# write the result on textfile
f = open(textfile_path, 'a', encoding='UTF-8')
for key, value in option_list.items():
  f.write(f'{value} ')
f.write(f'\n')
f.close()